# MASTERDATA CHECKER

##### GUIDE:

Just execute all the cells, on by one, and do what it says: Select instance, introduce user and password and click on login, etc.

Execute all the methods in the section "FUNCTIONS" and run the checker and visualizer at the end of the notebook to see how it works. (For the checker, you need to upload a file, just an upload button)

## LOGIN

Here just execute this cell to import all the needed classes and packages tht this tool includes. If some of the import fails, try to execute "pip install" + package_name (name of the package that failed, for example, pip install ipywidgets)

In [4]:
import openpyxl
import re
import pandas as pd
import csv
import os
import ipywidgets as widgets
import time
import tempfile
from datetime import datetime
from pybis import Openbis
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import getpass

### Select the instance

After executing the following cell (you don't need to change anything, just run it), a dropdown selector will appear, where you will need just to select the desired openbis instance, and continue to the next cell.

In [5]:
instance = widgets.Dropdown(
    options=['devel', 'main', 'schulung', 'test'],
    value='devel',
    description='Instance:',
    disabled=False,
)
display(instance)

Dropdown(description='Instance:', options=('devel', 'main', 'schulung', 'test'), value='devel')

### Enter username and password to login into selected openBIS instance

Same as before, just execute the following cell without touching the code, and a login widget will appear, where you will need to introduce your username and password, and then click in the "Login" button. If something fails, you can use the cell after this one to login in a dfferent way. If it works, "Login succesful!" will appear under the button.

In [11]:
usr = widgets.Text(
    value='',
    placeholder='Enter user name',
    description='User:',
    disabled=False   
)
psswd = widgets.Password(
    value='',
    placeholder='Enter password',
    description='Password:',
    disabled=False
)
output = widgets.Output()
url = f"https://{instance.value}.datastore.bam.de/"
o = Openbis(url)

def on_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        username = usr.value
        password = psswd.value
        o.login(username, password, save_token=True)
        if(o.is_session_active()):
            display(HTML(f"<p>Login successful!</p>"))
        else:
            display(HTML(f"<p>Login failed: {str(e)}</p>"))

button = widgets.Button(
    description="Login",
    button_style='success', 
    tooltip='Click to login in openBIS',
    icon='login' 
)

button.on_click(on_button_click)
display(usr)
display(psswd)
display(button)
display(output)

Text(value='', description='User:', placeholder='Enter user name')

Password(description='Password:', placeholder='Enter password')

Button(button_style='success', description='Login', icon='login', style=ButtonStyle(), tooltip='Click to login…

Output()

Just execute this cell if the above login code fails! Not needed if it worked!

In [12]:
username = usr.value
password = psswd.value

url = f"https://{instance.value}.datastore.bam.de/"
o = Openbis(url)
o.login(username, password, save_token=True)

'cmadaria-240730113617513x89F835FBF39DE458CA3AC4E3D443A558'

## FUNCTIONS

Here are the needed functions to execute the masterdata checker and visualizer. Execute all of them. It won't produce any output and the execution will be instant, imagine this as "registering" the functions. We will execute them later.

In [13]:
def name_checker(file_path):
    file_name = file_path.split("/")[-1]
    
    # Define the pattern for a valid file name
    pattern = r"^(collection_type|object_type|dataset_type|vocabulary)_([\w.]+)_(v\d+)_([a-zA-Z0-9]+(?:\.[0-9]+)?)_([a-zA-Z0-9]+)\.(xls|xlsx)$"
 
    # Check if the file name matches the pattern
    match = re.match(pattern, file_name)

    if match:
        # Extract parts of the file name
        entity_type, entity_name, version, division, contact_person, extension = match.groups()
        #print(entity_type, entity_name, version, division, contact_person, extension)
        return "File name: OK!"
    else:
        # Return specific errors and positions
        errors = []
        file_name = file_name.split(".xls")
        
        if len(file_name) < 2:
            errors.append("Invalid file format. Only .xls and .xlsx accepted")
            return errors
        
        else:
            file_parts = file_name[0].split("_")
            if len(file_parts) < 2:
                errors.append("Invalid name format. The name should contain different fields separated by underscores (_). Consult the wiki to see which ones.")
                return errors
            creator = file_parts.pop(-1)
            section = file_parts.pop(-1)
            version = file_parts.pop(-1)
            etype = file_parts.pop(0)
            if (etype == "object" or etype == "collection" or etype == "dataset"):
                etype = etype + "_" + file_parts.pop(0)
            code = "_".join(file_parts)
            
            if not re.match(r"^(collection_type|object_type|dataset_type|vocabulary)$", etype):
                errors.append("Invalid entity type at position 1.")
            if not re.match(r"^([\w.]+)$", code):
                errors.append("Invalid entity name at position 2.")
            if not re.match(r"^(v\d+)$", version):
                errors.append("Invalid version at position 3.")
            if not re.match(r"^([a-zA-Z0-9]+(?:\.[0-9]+)?)$", section):
                errors.append("Invalid division at position 4.")
            if not re.match(r"^[a-zA-Z0-9]+$", creator):
                errors.append("Invalid contact person at position 5.")
            
            return "\n".join(errors)

In [14]:
def index_to_excel_column(index):
    column = ''
    while index > 0:
        index, remainder = divmod(index - 1, 26)
        column = chr(65 + remainder) + column
    return column


def check_properties(sheet, errors):
    expected_terms = [
        "Version",
        "Code",
        "Description",
        "Mandatory",
        "Show in edit views",
        "Section",
        "Property label",
        "Data type",
        "Vocabulary code"
    ]
    row_headers = [cell.value for cell in sheet[4]]
    for term in expected_terms:
        if (term not in row_headers):
            if term in ("Mandatory","Show in edit views","Section"):
                errors.append(f"Warning: '{term}' not found in the properties headers.")
            else:
                errors.append(f"Error: '{term}' not found in the properties headers.")
        else:
             # Find the index of the term in the second row
             term_index = row_headers.index(term) + 1
             term_letter = index_to_excel_column(term_index)
             #print(term_index)
             
             # Check the column below "Version"
             if term == "Version":
                 column_below_version = []
                 for cell in sheet[term_letter][4:]:
                     if cell.value is not None:
                         column_below_version.append(cell.value)
                     else:
                         pass

                 # Check if any value in the column is not an integer
                 non_integer_indices = [i + 5 for i, cell in enumerate(column_below_version) if not (str(cell).isnumeric() or "$" in str(cell))]
                 if non_integer_indices:
                     # Append an error indicating the positions (row numbers) that are not integers
                     errors.append(f"Error: Values not valid found in the 'Version' column (they should be Integers) at row(s): {', '.join(map(str, non_integer_indices))}")

            # Check the column below "Code"
             elif term == "Code":
                column_below_code = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_code.append(cell.value)
                    else:
                        pass
                invalid_codes = [i + 5 for i, cell in enumerate(column_below_code) if not (re.match(r'^\$?[A-Z0-9_.]+$', str(cell)) or "$" in str(cell))]
                if invalid_codes:
                    # Append an error indicating the positions (row numbers) with invalid values for the current term
                    errors.append(f"Error: Invalid code found in the '{term}' column at row(s): {', '.join(map(str, invalid_codes))}")
                    
                #check that all the properties of the object are different using a set (unique terms):
                if len(set(column_below_code)) != len(column_below_code):
                    seen_props = set()
                    repeated_props = set()
                    for prop in column_below_code:
                        if prop in seen_props:
                            repeated_props.add(prop)
                        else:
                            seen_props.add(prop)
                    errors.append(f"Error: The following properties are repeated: {repeated_props}. Please, delete the duplicates, and leave just one occurence")

            
            
            # Check the cell below "Description"
             elif term == "Description":
                column_below_description = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_description.append(cell.value)
                    else:
                        pass
                invalid_indices = [i + 5 for i, cell in enumerate(column_below_description) if not (re.match(r'.*//.*', str(cell)) or "$" in str(cell))]
                if invalid_indices:
                    errors.append(f"Error: Invalid value(s) found in the '{term}' column at row(s): {', '.join(map(str, invalid_indices))}. Description should follow the schema: English Description + '//' + German Description.")

            # Check the cell below "Mandatory"
             elif term == "Mandatory":
                column_below_mandatory = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_mandatory.append(str(cell.value).upper())
                    else:
                        pass
                invalid_mandatory = [i + 5 for i, cell in enumerate(column_below_mandatory) if (cell not in ["TRUE", "FALSE"] and "$" not in str(cell))]
                if invalid_mandatory:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_mandatory))}. Accepted values: TRUE, FALSE")

            # Check the cell below "Show in edit views"
             elif term == "Show in edit views":
                column_below_show = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_show.append(str(cell.value).upper())
                    else:
                        pass
                invalid_show = [i + 5 for i, cell in enumerate(column_below_show) if (cell not in ["TRUE", "FALSE"] and "$" not in str(cell))]
                if invalid_show:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_show))}. Accepted values: TRUE, FALSE")

            # Check the cell below "Section"
             elif term == "Section":
                column_below_section = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_section.append(cell.value)
                    else:
                        pass
                invalid_section = [i + 5 for i, cell in enumerate(column_below_section) if not (re.match(r'.*', str(cell)) or "$" in str(cell))]
                if invalid_section:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_section))}. Specify the section as text format")

            # Check the cell below "Property label"
             elif term == "Property label":
                column_below_label = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_label.append(cell.value)
                    else:
                        pass
                invalid_label = [i + 5 for i, cell in enumerate(column_below_label) if not (re.match(r'.*', str(cell)) or "$" in str(cell))]
                if invalid_label:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_label))}. Specify the property label as text format")

            # Check the cell below "Data type"
             elif term == "Data type":
                column_below_type = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_type.append(str(cell.value).upper())
                    else:
                        pass
                invalid_type = [i + 5 for i, cell in enumerate(column_below_type) if (cell not in ["INTEGER", "REAL", "VARCHAR", "MULTILINE_VARCHAR", "HYPERLINK", "BOOLEAN", "CONTROLLEDVOCABULARY", "XML", "TIMESTAMP", "DATE", "SAMPLE"] and "$" not in str(cell))]
                if invalid_type:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_type))}. Accepted types: INTEGER, REAL, VARCHAR, MULTILINE_VARCHAR, HYPERLINK, BOOLEAN, CONTROLLEDVOCABULARY, XML, TIMESTAMP, DATE, SAMPLE")

            # Check the column below "Vocabulary code"
             elif term == "Vocabulary code":
                column_below_vocab = sheet[term_letter][4:]
                invalid_vocab = [i + 5 for i, cell in enumerate(column_below_vocab) if cell.value and not (re.match(r'^\$?[A-Z0-9_.]', str(cell.value)) or "$" not in str(cell))]
                if invalid_vocab:
                    # Append an error indicating the positions (row numbers) with invalid values for the current term
                    errors.append(f"Error: Invalid vocabulary code found in the '{term}' column at row(s): {', '.join(map(str, invalid_vocab))}")
    
    return errors

def check_vocab_terms(sheet, errors):
    expected_terms = [
        "Version",
        "Code",
        "Label"
        "Description"
    ]
    row_headers = [cell.value for cell in sheet[4]]
    for term in expected_terms:
        if term not in row_headers:
            errors.append(f"Error: '{term}' not found in the vocabulary term headers.")
        else:
             # Find the index of the term in the second row
             term_index = row_headers.index(term) + 1
             term_letter = index_to_excel_column(term_index)
             #print(term_index)
             
             # Check the column below "Version"
             if term == "Version":
                 column_below_version = []
                 for cell in sheet[term_letter][4:]:
                     if cell.value is not None:
                         column_below_version.append(cell.value)
                     else:
                         pass

                 # Check if any value in the column is not an integer
                 non_integer_indices = [i + 5 for i, cell in enumerate(column_below_version) if not str(cell).isnumeric()]
                 if non_integer_indices:
                     # Append an error indicating the positions (row numbers) that are not integers
                     errors.append(f"Error: Values not valid found in the 'Version' column (they should be Integers) at row(s): {', '.join(map(str, non_integer_indices))}")

            # Check the column below "Code"
             elif term == "Code":
                column_below_code = []
                for cell in sheet[term_letter][4:]:
                    if cell.value is not None:
                        column_below_code.append(cell.value)
                    else:
                        pass
                invalid_codes = [i + 5 for i, cell in enumerate(column_below_code) if not re.match(r'^\$?[A-Z0-9_.]+$', str(cell))]
                if invalid_codes:
                    # Append an error indicating the positions (row numbers) with invalid values for the current term
                    errors.append(f"Error: Invalid code found in the '{term}' column at row(s): {', '.join(map(str, invalid_codes))}")
                
                #check that all the properties of the object are different using a set (unique terms):
                if len(set(column_below_code)) != len(column_below_code):
                    seen_terms = set()
                    repeated_terms = set()
                    for term in column_below_code:
                        if term in seen_terms:
                            repeated_terms.add(term)
                        else:
                            seen_terms.add(term)
                    errors.append(f"Error: The following vocabulary terms are repeated: {repeated_terms}. Please, delete the duplicates, and leave just one occurence")

            
            
            # Check the cell below "Description"
             elif term == "Description":
                column_below_description = sheet[term_letter][4:]
                invalid_description = [i + 5 for i, cell in enumerate(column_below_description) if cell.value and not re.match(r'.*//.*', str(cell.value))]
                if invalid_description:
                    errors.append(f"Error: Invalid value(s) found in the '{term}' column at row(s): {', '.join(map(str, invalid_description))}. Description should follow the schema: English Description + '//' + German Description.")

            # Check the cell below "Mandatory"
             elif term == "Label":
                column_below_label = sheet[term_letter][4:]
                invalid_label = [i + 5 for i, cell in enumerate(column_below_label) if cell.value and not re.match(r'.*', str(cell.value))]
                if invalid_label:
                    errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_section))}. Specify the label as text format")
            
    return "\n".join(errors)

#file_path = 'C:/Users/cmadaria/Documents/Projects/Type checker/object_type_CHEMICAL_v1_S.3_relathma.xlsx'
def content_checker(file_path):
    workbook = openpyxl.load_workbook(file_path)
    errors = []
    file_name = file_path.split("/")[-1]
    file_name = file_name.split(".xls")
    file_parts = file_name[0].split("_")
    file_parts.pop(-1)
    file_parts.pop(-1)
    version = file_parts.pop(-1)
    etype = file_parts.pop(0)
    if (etype == "object" or etype == "collection" or etype == "dataset"):
        etype = etype + "_" + file_parts.pop(0)
    code = "_".join(file_parts)

    sheet = workbook.active
    
    filtered_rows = []
    
    for row in sheet.iter_rows(min_row=1, values_only=True):
    # Check if any cell in the row contains "$"
        if any("$" in str(cell) for cell in row):
            filtered_rows.append(["$" + str(cell) if cell is not None else None for cell in row])
        else:
            # If the row passed the check, add it to the filtered list
            filtered_rows.append(row)
    
    #remove all the rows in the sheet
    sheet.delete_rows(0, sheet.max_row)

    # Append the filtered rows to the sheet
    for row_data in filtered_rows:
        sheet.append(row_data)

    # Access a specific cell (e.g., cell A1)
    cell_value_A1 = sheet['A1'].value
    print(f"Entity Type: {cell_value_A1}")
    
    entity_types = ["SAMPLE_TYPE", "EXPERIMENT_TYPE", "DATASET_TYPE", "PROPERTY_TYPE", "VOCABULARY_TYPE"]
    if cell_value_A1 not in entity_types:
        errors.append("The entity type (cell A1) should be one of the following: SAMPLE_TYPE, EXPERIMENT_TYPE, DATASET_TYPE, PROPERTY_TYPE, VOCABULARY_TYPE")
        return "\n".join(errors)
    else:
        if cell_value_A1 == "SAMPLE_TYPE":
            expected_terms = [
                "Version",
                "Code",
                "Description",
                "Validation script",
                "Generated code prefix",
                "Auto generate codes",
            ]
            second_row_values = [cell.value for cell in sheet[2]]
            for term in expected_terms:
                if term not in second_row_values:
                    errors.append(f"Error: '{term}' not found in the entity headers.")
                else:
                     # Find the index of the term in the second row
                     term_index = second_row_values.index(term)

                     # Check the cell below "Version"
                     if term == "Version":
                        cell_below_version = sheet.cell(row=3, column=term_index + 1)
                        if str(cell_below_version.value) != version[1:]:
                            errors.append("Error: The version should be the same one indicated in the file name")

                    # Check the cell below "Code"
                     elif term == "Code":
                        cell_below_code = sheet.cell(row=3, column=term_index + 1)
                        if cell_below_code.value != code:
                            errors.append("Error: The code should be the same one indicated in the file name")
                    
                    
                    # Check the cell below "Description"
                     elif term == "Description":
                        cell_below_description = sheet.cell(row=3, column=term_index + 1)
                        description_pattern = re.compile(r".*//.*")
                        if not description_pattern.match(cell_below_description.value):
                            errors.append("Error: Description should follow the schema: English Description + '//' + German Description.")

                    # Check the cell below "Generated code prefix"
                     elif term == "Generated code prefix":
                        cell_below_generated_code = sheet.cell(row=3, column=term_index + 1)
                        code_replace = code.replace('_', '.').split('.')
                        ext_code = [word[:3].upper() for word in code_replace]
                        generated_code = '.'.join(ext_code)
                        if cell_below_generated_code.value != generated_code:
                            errors.append("Warning: It is recommended that the value of 'Generated code prefix' be the first three letters of each part of the 'Code' separated by dots ['.'].")

                    # Check the cell below "Validation script"
                     elif term == "Validation script":
                        cell_below_validation = sheet.cell(row=3, column=term_index + 1)
                        validation_pattern = re.compile(r"^[A-Za-z0-9_]+\.py$")
                        if cell_below_validation.value and not validation_pattern.match(cell_below_validation.value):
                             errors.append("Error: Validation script should follow the schema: Words and/or numbers separated by '_' and ending in '.py'")


                    # Check the cell below "Auto generate codes"
                     elif term == "Auto generate codes":
                        cell_below_auto_generate = sheet.cell(row=3, column=term_index + 1)
                        auto_code = cell_below_auto_generate.value
                        if (auto_code == True): auto_code = "TRUE"
                        if (auto_code == False): auto_code = "FALSE"
                        if auto_code not in ["TRUE", "FALSE"]:
                            errors.append("Error: Value below 'Auto generate codes' should be 'TRUE' or 'FALSE'.")
            
            errors = check_properties(sheet, errors)      
            
        elif cell_value_A1 == "EXPERIMENT_TYPE" or cell_value_A1 == "DATASET_TYPE":
            expected_terms = [
                "Version",
                "Code",
                "Description",
                "Validation script"
            ]
            second_row_values = [cell.value for cell in sheet[2]]
            for term in expected_terms:
                if term not in second_row_values:
                    errors.append(f"Error: '{term}' not found in the second row.")
                else:
                     # Find the index of the term in the second row
                     term_index = second_row_values.index(term)

                     # Check the cell below "Version"
                     if term == "Version":
                        cell_below_version = sheet.cell(row=3, column=term_index + 1)
                        if str(cell_below_version.value) != version[1:]:
                            errors.append("Error: The version should be the same one indicated in the file name")

                    # Check the cell below "Code"
                     elif term == "Code":
                        cell_below_code = sheet.cell(row=3, column=term_index + 1)
                        if cell_below_code.value != code:
                            errors.append("Error: The code should be the same one indicated in the file name")
                    
                    
                    # Check the cell below "Description"
                     elif term == "Description":
                        cell_below_description = sheet.cell(row=3, column=term_index + 1)
                        description_pattern = re.compile(r".*//.*")
                        if not description_pattern.match(cell_below_description.value):
                            errors.append("Error: Description should follow the schema: English Description + '//' + German Description.")
            
            
                    # Check the cell below "Validation script"
                     elif term == "Validation script":
                        cell_below_validation = sheet.cell(row=3, column=term_index + 1)
                        validation_pattern = re.compile(r"^[A-Za-z0-9_]+\.py$")
                        if cell_below_validation.value and not validation_pattern.match(cell_below_validation.value):
                            errors.append("Error: Validation script should follow the schema: Words and/or numbers separated by '_' and ending in '.py'")

            errors = check_properties(sheet, errors) 
            
        elif cell_value_A1 == "VOCABULARY_TYPE":
            expected_terms = [
                "Version",
                "Code",
                "Description"
            ]
            second_row_values = [cell.value for cell in sheet[2]]
            for term in expected_terms:
                if term not in second_row_values:
                    errors.append(f"Error: '{term}' not found in the second row.")
                else:
                     # Find the index of the term in the second row
                     term_index = second_row_values.index(term)

                     # Check the cell below "Version"
                     if term == "Version":
                        cell_below_version = sheet.cell(row=3, column=term_index + 1)
                        if str(cell_below_version.value) != version[1:]:
                            errors.append("Error: The version should be the same one indicated in the file name. Value found: {cell_below_version.value}")

                    # Check the cell below "Code"
                     elif term == "Code":
                        cell_below_code = sheet.cell(row=3, column=term_index + 1)
                        if cell_below_code.value != code:
                            errors.append("Error: The code should be the same one indicated in the file name. Value found: {cell_below_code.value}")
                    
                    
                    # Check the cell below "Description"
                     elif term == "Description":
                        cell_below_description = sheet.cell(row=3, column=term_index + 1)
                        description_pattern = re.compile(r".*//.*")
                        if not description_pattern.match(cell_below_description.value):
                            errors.append("Error: Description should follow the schema: English Description + '//' + German Description. Value found: {cell_below_description.value}")
            
            errors = check_vocab_terms(sheet, errors)

        elif cell_value_A1 == "PROPERTY_TYPE":
            expected_terms = [
                "Version",
                "Code",
                "Description",
                "Mandatory",
                "Show in edit views",
                "Section",
                "Property label",
                "Data type",
                "Vocabulary code"
            ]
            second_row_values = [cell.value for cell in sheet[2]]
            for term in expected_terms:
                if term not in second_row_values:
                    errors.append(f"Error: '{term}' not found in the second row.")
                else:
                     # Find the index of the term in the second row
                     term_index = second_row_values.index(term) + 1


                     # Check the column below "Version"
                     if term == "Version":
                        column_below_version = sheet[term_index][2:]
                        # Check if any value in the column is not an integer
                        non_integer_indices = [i + 3 for i, cell in enumerate(column_below_version) if not isinstance(cell.value, int)]
                        if non_integer_indices:
                            # Append an error indicating the positions (row numbers) that are not integers
                            errors.append(f"Error: Values not valid found in the 'Version' column (they should be Integers) at row(s): {', '.join(map(str, non_integer_indices))}. Value found: {cell.value}")

                    # Check the column below "Code"
                     elif term == "Code":
                        column_below_code = sheet[term_index][2:]
                        invalid_codes = [i + 3 for i, cell in enumerate(column_below_code) if not re.match(r'^\$?[A-Z0-9_.]+$', str(cell.value))]
                        if invalid_codes:
                            # Append an error indicating the positions (row numbers) with invalid values for the current term
                            errors.append(f"Error: Invalid code found in the '{term}' column at row(s): {', '.join(map(str, invalid_codes))}. Value found: {cell.value}")
                    
                    
                    # Check the cell below "Description"
                     elif term == "Description":
                        column_below_description = sheet[term_index][2:]
                        invalid_indices = [i + 3 for i, cell in enumerate(column_below_code) if not re.match(r'.*//.*', str(cell.value))]
                        if invalid_indices:
                            errors.append(f"Error: Invalid value(s) found in the '{term}' column at row(s): {', '.join(map(str, invalid_indices))}. Description should follow the schema: English Description + '//' + German Description. Value found: {cell.value}")

                    # Check the cell below "Mandatory"
                     elif term == "Mandatory":
                        column_below_mandatory = sheet[term_index][2:]
                        invalid_mandatory = [i + 3 for i, cell in enumerate(column_below_mandatory) if cell.value not in ["TRUE", "FALSE"]]
                        if invalid_mandatory:
                            errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_mandatory))}. Accepted values: TRUE, FALSE. Value found: {cell.value}")

                    # Check the cell below "Show in edit views"
                     elif term == "Show in edit views":
                        column_below_show = sheet[term_index][2:]
                        invalid_show = [i + 3 for i, cell in enumerate(column_below_show) if cell.value not in ["TRUE", "FALSE"]]
                        if invalid_show:
                            errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_show))}. Accepted values: TRUE, FALSE. Value found: {cell.value}")

                    # Check the cell below "Section"
                     elif term == "Section":
                        column_below_section = sheet[term_index][2:]
                        invalid_section = [i + 3 for i, cell in enumerate(column_below_section) if not re.match(r'.*', str(cell.value))]
                        if invalid_section:
                            errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_section))}. Specify the section as text format. Value found: {cell.value}")

                    # Check the cell below "Property label"
                     elif term == "Property label":
                        column_below_label = sheet[term_index][2:]
                        invalid_label = [i + 3 for i, cell in enumerate(column_below_label) if not re.match(r'.*', str(cell.value))]
                        if invalid_label:
                            errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_label))}. Specify the property label as text format. Value found: {cell.value}")

                    # Check the cell below "Data type"
                     elif term == "Data type":
                        column_below_type = sheet[term_index][2:]
                        invalid_type = [i + 3 for i, cell in enumerate(column_below_type) if cell.value not in ["INTEGER", "REAL", "VARCHAR", "MULTILINE_VARCHAR", "HYPERLINK", "BOOLEAN", "CONTROLLEDVOCABULARY", "XML", "TIMESTAMP", "DATE", "SAMPLE"]]
                        if invalid_type:
                            errors.append(f"Error: Invalid value found in the '{term}' column at row(s): {', '.join(map(str, invalid_type))}. Accepted types: INTEGER, REAL, VARCHAR, MULTILINE_VARCHAR, HYPERLINK, BOOLEAN, CONTROLLEDVOCABULARY, XML, TIMESTAMP, DATE, SAMPLE.  Value found: {cell.value}")

                    # Check the column below "Vocabulary code"
                     elif term == "Vocabulary code":
                        column_below_vocab = sheet[term_index][2:]
                        invalid_vocab = [i + 3 for i, cell in enumerate(column_below_vocab) if cell.value is not None and not re.match(r'^\$?[A-Z0-9_.]+$', str(cell.value))]
                        if invalid_vocab:
                            # Append an error indicating the positions (row numbers) with invalid values for the current term
                            errors.append(f"Error: Invalid vocabulary code found in the '{term}' column at row(s): {', '.join(map(str, invalid_vocab))}. Value found: {cell.value}")


    # Close the workbook after use
    workbook.close()
    output = "\n".join(errors)
    if output == "":
        return "File content: OK!"
    else:
        return output

In [15]:
def search_entity(o, e_type, e_code):
    if e_type == "EXPERIMENT_TYPE":
        return o.get_collection_type(e_code)
        
    elif e_type == "SAMPLE_TYPE":
        return o.get_object_type(e_code)
    
    elif e_type == "DATASET_TYPE":
        return o.get_dataset_type(e_code)
    
def get_entity_list(o, entity_type):
    if entity_type == "EXPERIMENT_TYPE":
        return o.get_collection_types()
    
    elif entity_type == "SAMPLE_TYPE":
        return o.get_object_types()
    
    elif entity_type == "DATASET_TYPE":
        return o.get_dataset_types()
    
def compare_objects(obj1, obj2):
    # Check if both are None or both are empty strings
    if (obj1 is None and obj2 == "") or (obj1 == "" and obj2 is None):
        return True
    elif (obj1 == "False" and obj2 == "FALSE") or (obj1 == "FALSE" and obj2 == "False"):
        return True
    elif (obj1 == "True" and obj2 == "TRUE") or (obj1 == "TRUE" and obj2 == "True"):
        return True
    else:
        return obj1 == obj2
    
def get_df_value(df, prop, attr):
    column_name = 'propertyType'
    value_to_find = prop
    
    # Create a boolean mask for rows where the condition is met
    mask = df[column_name] == value_to_find

    # Use the boolean mask to filter the DataFrame
    filtered_df = df[mask]

    column_to_access = attr
    
    # Using .at[] for a specific row
    value = filtered_df.at[filtered_df.index[0], column_to_access]
    
    return value
    


def check_entity_same_code(file_path, o, openbis_entity):
    errors = []
    description = ""
    auto_code = ""
    val_script = ""
    prefix_code = ""

    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    entity_type = sheet['A1'].value
    second_row_values = [cell.value for cell in sheet[2]]
    
    for term in second_row_values:
        term_index = second_row_values.index(term)
        if term == "Code":
            entity_code = sheet.cell(row=3, column=term_index + 1).value
        elif term == "Description":
            description = sheet.cell(row=3, column=term_index + 1).value
        elif term == "Auto generate codes":
            auto_code = sheet.cell(row=3, column=term_index + 1).value
        elif term == "Validation script":
            val_script = sheet.cell(row=3, column=term_index + 1).value
        elif term == "Generated code prefix":
            prefix_code = sheet.cell(row=3, column=term_index + 1).value
            
        #format values to match
        if (auto_code == True): auto_code = "TRUE"
        if (auto_code == False): auto_code = "FALSE"
        if (val_script == None): val_script = ""
    
    
    openbis_description = openbis_entity.description
    openbis_auto_code = openbis_entity.autoGeneratedCode
    openbis_val_script = openbis_entity.validationPlugin
    openbis_prefix_code = openbis_entity.generatedCodePrefix
        
    #cast values to STRING t match Excel data
    if (openbis_auto_code == True): openbis_auto_code = "TRUE"
    if (openbis_auto_code == False): openbis_auto_code = "FALSE"
    if (openbis_val_script == None): openbis_val_script = ""
        
    #check description
    if (description != openbis_description):
        errors.append(f"The Description of ('{entity_type}') '{entity_code}' has been changed compared to the previous version.")
        description_pattern = re.compile(r".*//.*")
        if not description_pattern.match(description):
            errors.append("Error: The Description of ('{entity_type}') '{entity_code}' should follow the schema: English Description + '//' + German Description.")
        
    #check auto-generated codes
    if (auto_code != openbis_auto_code):
        errors.append(f"The value of “Auto generate codes” of ('{entity_type}') '{entity_code}' has been changed from '{openbis_auto_code}' to '{auto_code}'.")
    
    #check validation scripts
    if (val_script == "" and openbis_val_script != ""):
        errors.append(f"The validation script '{openbis_val_script}' has been deleted from ('{entity_type}') '{entity_code}'.")
    elif (val_script != "" and openbis_val_script == ""):
        errors.append(f"A validation script '{val_script}' has been added to ('{entity_type}') '{entity_code}'.")
    elif (val_script != openbis_val_script):
        errors.append(f"The validation script of ('{entity_type}') '{entity_code}' has been changed from '{openbis_val_script}' to '{val_script}'.")
        
    #check generated code prefix
    if (prefix_code != openbis_prefix_code):
        errors.append(f"The Code Prefix of ('{entity_type}') '{entity_code}' has been changed from '{openbis_prefix_code}' to '{prefix_code}'.")
            
            
    #get assigned properties from the excel file
    prop_headers = [cell.value for cell in sheet[4]]
    entity_properties = []
    term_index = prop_headers.index("Code") + 1
    term_letter = index_to_excel_column(term_index)
        
    for cell in sheet[term_letter][4:]:
        if cell.value is not None:
            entity_properties.append(cell.value)
        
    #get assigned properties from the openbis instance
    openbis_entity_properties = []
    for prop in openbis_entity.get_property_assignments():
        openbis_entity_properties.append(prop.permId)
            
    #check if the properties lists are the same
    if sorted(entity_properties) != sorted(openbis_entity_properties):
        errors.append(f"The set of Property Types assigned to the ('{entity_type}') '{entity_code}' has been changed compared to the previous version.")

            
    #check which properties has been added and removed
    deleted_properties = []
    added_properties = []
        
    deleted_properties = list(set(openbis_entity_properties) - set(entity_properties))
    added_properties = list(set(entity_properties) - set(openbis_entity_properties))
        
    for d_prop in deleted_properties:
        errors.append(f"The Property type assignment '{d_prop}' has been removed.")
    for a_prop in added_properties:
        errors.append(f"The Property type assignment '{a_prop}' has been added.")

    #save dict with all the properties values from the entity in the instance
    openbis_properties_data = {}
    for prop in openbis_entity.get_property_assignments():
        openbis_properties_data[prop.code] = {
            "label": prop.label,
            "description": prop.description,
            "dataType": prop.dataType,
            "vocabulary": prop.vocabulary if prop.vocabulary is not None else "",
            "metaData" : prop.metaData
        }
            
    #save dict with all the properties values from the excel metadata file
    prop_headers = [cell.value for cell in sheet[4]]
    properties_data = {}
    term_index = prop_headers.index("Code") + 1
    term_letter = index_to_excel_column(term_index)
        

    for row in sheet.iter_rows(min_row=5, values_only=True):
        code_value = row[term_index - 1]  # Index is 0-based
        if code_value is not None:

            properties_data[code_value] = {
                'label': row[prop_headers.index('Property label')],
                'description': row[prop_headers.index('Description')],
                'dataType': row[prop_headers.index('Data type')],
                "vocabulary": row[prop_headers.index('Vocabulary code')] if row[prop_headers.index('Vocabulary code')] is not None else "",
                'metaData': {} if row[prop_headers.index('Metadata')] in (None, "") else row[prop_headers.index('Metadata')],
                'mandatory': row[prop_headers.index('Mandatory')],
                'section': row[prop_headers.index('Section')],
                'plugin': row[prop_headers.index('Dynamic script')],
                }

    assigned_properties = openbis_entity.get_property_assignments().df
    #properties present in the excel but not in openbis: not assigned
    not_assigned_properties =  set(properties_data.keys()) - set(openbis_properties_data.keys())
    
    #compare both dicts with sets of properties to check the differences
    for key in openbis_properties_data.keys() & properties_data.keys():
        for assigned_field in ["mandatory", "section", "plugin"]:
            excel_assigned = properties_data[key][assigned_field]
            openbis_assigned = get_df_value(assigned_properties, key, assigned_field)
            if not compare_objects(excel_assigned,openbis_assigned):
                if assigned_field == "mandatory":
                    if (str(openbis_assigned).upper() == "FALSE" and str(excel_assigned).upper() == "TRUE"):
                        errors.append(f"The value of the attribute 'Mandatory' of Property type {key} has been changed compared to the previous version from FALSE to TRUE.")
                    elif (str(openbis_assigned).upper() == "TRUE" and str(excel_assigned).upper() == "FALSE"):
                        errors.append(f"ERROR: The value of the attribute 'Mandatory' of Property type {key} has been changed compared to the previous version from TRUE to FALSE. This is NOT allowed")
                elif assigned_field == "section":
                    errors.append(f"The section of Property type {key} has been changed compared to the previous version from {openbis_assigned} to {excel_assigned}.")
                elif assigned_field == "plugin":
                    if (openbis_assigned == "" or openbis_assigned == None) and (excel_assigned != "" or excel_assigned != None):
                        errors.append(f"WARNING: A dynamic property script ({excel_assigned}) has been added retrospectively to the Property type {key}.")
                    elif (str(openbis_assigned).upper() != str(excel_assigned).upper()):
                        errors.append(f"ERROR: The dynamic property script of Property type {key} has been changed or deleted compared to the previous version. This is NOT allowed")
                   
        for field in ["label", "description", "dataType", "vocabulary", "metaData"]:
            value1 = openbis_properties_data[key][field]
            value2 = properties_data[key][field]
            if not compare_objects(value1,value2):
                if field == "label":
                    errors.append(f"The label of Property type {key} has been changed compared to the previous version from {value1} to {value2}.")
                elif field == "description":
                    errors.append(f"The description of Property type {key} has been changed compared to the previous version from {value1} to {value2}.")
                elif field == "dataType":
                    errors.append(f"WARNING: The data type of Property type {key} has been changed compared to the previous version from from {value1} to {value2}. This is only permissible for some cases, e.g., 'CONTROLLEDVOCABULARY' to 'VARCHAR'!")
                elif field == "vocabulary":
                    errors.append(f"ERROR: The vocabulary code of Property type {key} has been changed compared to the previous version from from {value1} to {value2}. This is not allowed.")
                elif field == "metaData":
                    errors.append(f"ERROR: The metadata of Property type {key} has been changed compared to the previous version from from {value1} to {value2}. This is not allowed.")


    for key in not_assigned_properties:
        try:
             prop_ob = o.get_property_type(key)
             if not compare_objects(properties_data[key]['label'],prop_ob.label):
                 errors.append(f"The label of Property type {key} has been changed compared to the previous version from {prop_ob.label} to {properties_data[key]['label']}.")
             elif not compare_objects(properties_data[key]['description'],prop_ob.description):
                 errors.append(f"The description of Property type {key} has been changed compared to the previous version from {prop_ob.description} to {properties_data[key]['description']}.")
             elif not compare_objects(properties_data[key]['dataType'],prop_ob.dataType):
                 errors.append(f"The data type of Property type {key} has been changed compared to the previous version from {prop_ob.dataType} to {properties_data[key]['dataType']}. This is only permissible for some cases, e.g., 'CONTROLLEDVOCABULARY' to 'VARCHAR'!")
             elif not compare_objects(properties_data[key]['vocabulary'],prop_ob.vocabulary):
                 errors.append(f"The vocabulary code of Property type {key} has been changed compared to the previous version from {prop_ob.vocabulary} to {properties_data[key]['vocabulary']}. This is not allowed.")
             elif not compare_objects(properties_data[key]['metaData'],prop_ob.metaData):
                 errors.append(f"The metadata of Property type {key} has been changed compared to the previous version from {prop_ob.metaData} to {properties_data[key]['metaData']}. This is not allowed.")
        except ValueError:
             continue
        
    workbook.close()
    
    return "\n".join(errors)
        
def check_entity_diff_code(file_path, o):
    errors = []
    
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    entity_type = sheet['A1'].value
    
    openbis_entity_types = get_entity_list(o, entity_type)
    
    openbis_entity_properties = {}
    
    #get all the properties for each entity type from the instance, and save them in a dictionary
    for etype in openbis_entity_types:
        props_by_type = []
        openbis_entity_properties[etype.code] = []
        if etype.code != "UNKNOWN":
            for prop in etype.get_property_assignments():
                props_by_type.append(prop.permId)
            openbis_entity_properties[etype.code] = props_by_type
    
    #get the assigned properties of the entity in the excel
    entity_headers = [cell.value for cell in sheet[2]]
    entity_properties = []
    term_index = entity_headers.index("Code") + 1
    entity_code = sheet.cell(row=3, column=term_index).value
    term_letter = index_to_excel_column(term_index)
    
    for cell in sheet[term_letter][4:]:
        if cell.value is not None:
            entity_properties.append(cell.value)
            
    for key, prop_list in openbis_entity_properties.items():
        if set(prop_list) == set(entity_properties):
            errors.append(f"The {entity_type} '{entity_code}' is very similar to the existing {entity_type} '{key}'. Please consider whether you need to create a new entity type or whether you can re-use '{key}'")
    
    return "\n".join(errors)


def check_prefix_sufix(file_path, o):
    errors = []
    
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    entity_type = sheet['A1'].value
    
    entity_headers = [cell.value for cell in sheet[2]]
    term_index = entity_headers.index("Code") + 1
    entity_code = sheet.cell(row=3, column=term_index).value
    
    pattern = re.compile(r'^[A-Za-z0-9_.]+\.[A-Za-z0-9_]+$')
    
    if pattern.match(entity_code):
        parts = entity_code.rsplit('.', 1)
        prefix = parts[0]
        
        prop_headers = [cell.value for cell in sheet[4]]
        entity_properties = []
        term_index = prop_headers.index("Code") + 1
        term_letter = index_to_excel_column(term_index)
        
        for cell in sheet[term_letter][4:]:
            if cell.value is not None:
                entity_properties.append(cell.value)
        
        #get assigned properties from the openbis instance
        try:
            prefix_entity = search_entity(o, entity_type, prefix)
        except ValueError as e:
            errors.append(f"Entity type '{prefix}' is not present in the system, and cannot be the prefix of a new entity to be registered.")
            return "\n".join(errors)
        
        prefix_properties = []
        for prop in prefix_entity.get_property_assignments():
            prefix_properties.append(prop.permId)
            
        #get the properties that are in the PREFIX but not in the SUFIX
        difference = [value for value in prefix_properties if value not in entity_properties]
        
        prefix_properties_data = {}
        for prop in prefix_entity.get_property_assignments():
            prefix_properties_data[prop.code] = {
                "label": prop.label,
                "description": prop.description,
                "dataType": prop.dataType,
                "vocabulary": prop.vocabulary if prop.vocabulary is not None else "",
                "metaData" : prop.metaData
            }
            
        entity_properties_data = {}
        for row in sheet.iter_rows(min_row=5, values_only=True):
            code_value = row[term_index - 1]  # Index is 0-based
            if code_value is not None:
                entity_properties_data[code_value] = {
                    'label': row[prop_headers.index('Property label')],
                    'description': row[prop_headers.index('Description')],
                    'dataType': row[prop_headers.index('Data type')],
                    "vocabulary": row[prop_headers.index('Vocabulary code')] if row[prop_headers.index('Vocabulary code')] is not None else "",
                    'metaData': {} if row[prop_headers.index('Metadata')] in (None, "") else row[prop_headers.index('Metadata')],
                    }

        changes = []
        #compare both dicts with sets of properties to check the differences
        for key in prefix_properties_data.keys() & entity_properties_data.keys():
            for field in ["label", "description", "dataType", "vocabulary", "metaData"]:
                value1 = prefix_properties_data[key][field]
                value2 = entity_properties_data[key][field]
                if value1 != value2:
                    if field == "label":
                        changes.append(f"Change in label of Property type {key}.")
                    elif field == "description":
                        changes.append(f"Change in description of Property type {key}.")
                    elif field == "dataType":
                        changes.append(f"Change in data type of Property type {key}.")
                    elif field == "vocabulary":
                        changes.append(f"Change in vocabulary code of Property type {key}.")
                    elif field == "metaData":
                        changes.append(f"Change in metadata of Property type {key}.")

        if (len(difference) != 0) or (len(changes) != 0):
            errors.append(f"As a specification of the entity type {prefix}, the entity type {entity_code} must include all Property types of {prefix} without any changes.")
            errors.append(f"The missing properties are: ")
            missing = ", ".join(difference)
            errors.append(missing)
            errors.append("\n")
            errors.append(f"The changed property attributes are: ")
            changed = "\n".join(changes)
            errors.append(changed)
    
            
        check_prefix_prefix(o, prefix, entity_type, errors)
    
    
    return "\n".join(errors)


def check_prefix_prefix(o, prefix, entity_type, errors):
    if '.' in prefix:
        # Split the string by the last dot
        prefix_2, suffix = prefix.rsplit('.', 1)
        
        prefix_entity = search_entity(o, entity_type, prefix_2)
        suffix_entity = search_entity(o, entity_type, suffix)
        
        prefix_properties = []
        for prop in prefix_entity.get_property_assignments():
            prefix_properties.append(prop.permId)
            
        suffix_properties = []
        for prop in suffix_entity.get_property_assignments():
            suffix_properties.append(prop.permId)
            
        difference = [value for value in prefix_properties if value not in suffix_properties]
        
        prefix_properties_data = {}
        for prop in prefix_entity.get_property_assignments():
            prefix_properties_data[prop.code] = {
                "label": prop.label,
                "description": prop.description,
                "dataType": prop.dataType,
                "vocabulary": prop.vocabulary if prop.vocabulary is not None else "",
                "metaData" : prop.metaData
            }
            
        suffix_properties_data = {}
        for prop2 in suffix_entity.get_property_assignments():
            suffix_properties_data[prop2.code] = {
                "label": prop2.label,
                "description": prop2.description,
                "dataType": prop2.dataType,
                "vocabulary": prop2.vocabulary if prop2.vocabulary is not None else "",
                "metaData" : prop2.metaData
            }
            
        changes = []
        #compare both dicts with sets of properties to check the differences
        for key in prefix_properties_data.keys() & suffix_properties_data.keys():
            for field in ["label", "description", "dataType", "vocabulary", "metaData"]:
                value1 = prefix_properties_data[key][field]
                value2 = suffix_properties_data[key][field]
                if value1 != value2:
                    if field == "label":
                        changes.append(f"Change in label of Property type {key}.")
                    elif field == "description":
                        changes.append(f"Change in description of Property type {key}.")
                    elif field == "dataType":
                        changes.append(f"Change in data type of Property type {key}.")
                    elif field == "vocabulary":
                        changes.append(f"Change in vocabulary code of Property type {key}.")
                    elif field == "metaData":
                        changes.append(f"Change in metadata of Property type {key}.")

        if (len(difference) != 0) or (len(changes) != 0):
            errors.append(f"As a specification of the entity type {prefix}, the entity type {entity_code} must include all Property types of {prefix} without any changes.")
            missing = ", ".join(difference)
            if missing != "":
                errors.append(f"The missing properties are: ")
                errors.append(missing)
            else:
                errors.append(f"There are no missing properties")
            errors.append(f"The changed property attributes are: ")
            changed = "\n".join(changes)
            errors.append(changed)


        # Recursively call the function with the prefix
        check_prefix_prefix(o, prefix_2, entity_type, errors)
        
        
def entity_checker(file_path):
    errors = []
    
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    entity_type = sheet['A1'].value
    entity_headers = [cell.value for cell in sheet[2]]
    term_index = entity_headers.index("Code") + 1
    entity_code = sheet.cell(row=3, column=term_index).value
    
    try:
        openbis_entity = search_entity(o, entity_type, entity_code)
    except ValueError as e:
        errors.append(f"Entity type '{entity_code}' is a new entity type (not present in the system) to be registered.")
        openbis_entity = ""
        
    if (openbis_entity != ""):
        errors.append(f"Entity type '{entity_code}' already exists.")
        same_code_errors = check_entity_same_code(file_path, o, openbis_entity)
        errors.append(same_code_errors)
    else:
        diff_code_errors = check_entity_diff_code(file_path, o)
        errors.append(diff_code_errors)
        
    prefix_errors = check_prefix_sufix(file_path, o)
    errors.append(prefix_errors)
    
    
    return "\n".join(errors)

In [16]:
def generate_csv_and_download():
    url = o.url
    
    header = ["INSTANCE", "DATE"]
    
    instance = url.split("//")[1].split(".")[0]
    
    current_date = datetime.now().strftime("%d-%m-%Y")
    
    info = [instance, current_date]

    print(f"Checking contents in instance: {instance}\n")
    
    # Fetch data from the server
    spaces = []
    projects = []
    experiment_types = []
    object_types = []
    material_types = []
    dataset_types = []
    vocabs = []
    plugins = []
    
    print(f"Listing SPACES in {instance}\n")
    print(f"Total number of SPACES: {str(o.get_spaces().totalCount)}\n")

    for space in o.get_spaces():
        print(f"  {space}")
        spaces.append(space)

    print(f"\nListing PROJECTS in {instance}\n")
    print(f"Total number of PROJECTS: {str(o.get_projects().totalCount)}\n")

    for project in o.get_projects():
        print(f"  {project.code}")
        projects.append(project.code)

    print(f"\nListing EXPERIMENT TYPES in {instance}\n")
    print(f"Total number of EXPERIMENT TYPES: {str(o.get_experiment_types().totalCount)}\n")
    
    for exp in o.get_experiment_types():
        print(f"  {exp}")
        experiment_types.append(exp)
    
    print(f"\nListing OBJECT TYPES in {instance}\n")
    print(f"Total number of OBJECT TYPES: {str(o.get_object_types().totalCount)}\n")

    objs = []
    for obj in o.get_object_types():
        objs.append(obj)
        print(f"  {obj}")
        if obj.code != "UNKNOWN":
            object_types.append(obj)

    print(f"\nListing MATERIAL TYPES in {instance}\n")
    print(f"Total number of MATERIAL TYPES: {str(o.get_material_types().totalCount)}\n")
    
    for material in o.get_material_types():
        print(f"  {material}")
        material_types.append(material)
    
    print(f"\nListing DATASET TYPES in {instance}\n")
    print(f"Total number of DATASET TYPES: {str(o.get_dataset_types().totalCount)}\n")
    
    for dataset in o.get_dataset_types():
        print(f"  {dataset}")
        dataset_types.append(dataset)
        
    print(f"\nListing VOCABULARIES in {instance}\n")
    print(f"Total number of VOCABULARIES: {str(o.get_vocabularies().totalCount)}\n")
    
    for vocab in o.get_vocabularies():
        print(f"  {vocab.code}")
        vocabs.append(vocab.code)
        
    print(f"\nListing PLUGINS in {instance}\n")
    print(f"Total number of PLUGINS: {str(o.get_plugins().totalCount)}\n")
    
    for plug in o.get_plugins():
        print(f"  {plug.name}")
        plugins.append(plug.name)


    masterdata_headers = [f"SPACES ({len(spaces)})", f"PROJECTS ({len(projects)})", f"EXPERIMENT TYPES ({len(experiment_types)})", 
                          f"OBJECT TYPES ({len(object_types)})", f"DATASET TYPES ({len(dataset_types)})",
                          f"VOCABULARIES ({len(vocabs)})", f"PLUGINS ({len(plugins)})", f"MATERIAL TYPES ({len(material_types)})"]
    
    
    # Combine master data into a list of lists
    masterdata = [
        spaces,
        projects,
        experiment_types,
        object_types,
        dataset_types,
        vocabs,
        plugins,
        material_types
    ]

    # Directory name based on instance
    directory = f"{instance}_data"
    
    # Create directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # File path
    filename = os.path.join(directory, f"{instance}_{datetime.now().strftime('%d%m%Y')}.csv")

    
    # Write data to CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the instance and date headers
        writer.writerow(["INSTANCE", "DATE"])
        
        # Write the instance and date info
        writer.writerow(info)
        
        # Write empty row
        writer.writerow("")
        
        # Write the master data headers
        writer.writerow(masterdata_headers)
        
        # Determine the maximum length of the master data lists
        max_length = max(len(data) for data in masterdata)
        
        # Write the master data vertically
        for i in range(max_length):
            row = []
            for data in masterdata:
                if i < len(data):
                    row.append(data[i])
                else:
                    row.append("")  # Append empty string if the list is shorter
            writer.writerow(row)
            
        # Write empty row
        writer.writerow("")
        
        writer.writerow(["PROPERTY LIST BY OBJECT TYPE"])
            
        # Write another header row with the content of object_types horizontally
        writer.writerow(object_types)
        
        props_by_obj = []
        
        for obj in object_types:
            if obj.code == "UNKNOWN":
                continue
            print(f"\nPROPERTY LIST for OBJECT {obj.code}\n")
            props = []
            for prop in obj.get_property_assignments():
                print(f"{prop.code} --> {str(prop.dataType).lower()}")
                props.append(f"{prop.code} ({str(prop.dataType).lower()})")
            props_by_obj.append(props)
            
        # Determine the maximum length of the object properties
        max_length_props = max(len(properties) for properties in props_by_obj)
        
        # Write the master data vertically
        for i in range(max_length_props):
            row = []
            for prop_list in props_by_obj:
                if i < len(prop_list):
                    row.append(prop_list[i])
                else:
                    row.append("")  # Append empty string if the list is shorter
            writer.writerow(row)
        
    return f"\nCSV file '{filename}' has been created."

## USE THE CHECKER

Once executed all the function cells above, here we continue with the checker, where we will need to upload a file, and run the checker with that file against the selected instance in the beginning. If you just want to run the Masterdata Visualizer, you can ignore this part and go directly to the "USE THE VISUALIZER" cells.

### UPLOAD THE EXCEL FILE

Execute this cell below, and a button to upload a file will appear. Then, after clicking on the button, a window for selecting the desired file will appear. Once that you selected the file, you can continue to run the checker in the next section.

In [18]:
uploader = widgets.FileUpload(
    accept='.xlsx',
    multiple=False
)

def save_uploaded_file(uploader):
    # Get the first uploaded file (since multiple=False, we expect only one file)
    uploaded_file = uploader.value[0]
    content = uploaded_file['content'].tobytes()
    filename = uploaded_file['name']
    
    # Create a temporary file to save the uploaded content
    temp_dir = tempfile.gettempdir()
    temp_file_path = os.path.join(temp_dir, filename)
    
    # Write the content to the temporary file
    with open(temp_file_path, 'wb') as f:
        f.write(content)
    
    return temp_file_path

display(uploader)

FileUpload(value=(), accept='.xlsx', description='Upload')

### RUN THE CHECKER

Here just run the cell below, and the Masterdata Checker will start to run. A loading bar will appear with the different procesess, and once that it finishes, all the checks will appear below.

In [19]:
if uploader.value:
    file_path = save_uploaded_file(uploader)
    print(f"File saved to: {file_path}")
else:
    print("No file uploaded yet.")

file_name = file_path.split("\\")[-1]

# Initialize tqdm
tasks = [
    {"name": "Name Check", "func": name_checker, "args": (file_name,)},
    {"name": "Content Check", "func": content_checker, "args": (file_path,)},
    {"name": "Entity Check", "func": entity_checker, "args": (file_path,)}
]

with tqdm(total=len(tasks), desc="Overall Progress", unit="task") as pbar:
    result_name = str(name_checker(file_name))
    pbar.update(1)

    if result_name != "File name: OK!":
        result_format = "\nNAME CHECKS:" + "\n-------------\n" + result_name
    else:
        result_content = str(content_checker(file_path))
        pbar.update(1)
        result_entity = str(entity_checker(file_path))
        pbar.update(1)
        result_format = "\nNAME CHECKS:" + "\n-------------\n" + result_name + "\n" + "\nCONTENT CHECKS:" + "\n-------------\n" + result_content + "\n" + "\nENTITY CHECKS" + "\n-------------\n" + result_entity

print(result_format)

File saved to: C:\Users\cmadaria\AppData\Local\Temp\object_type_CHEMICAL_v1_S.3_cmadaria.xlsx


Overall Progress:   0%|          | 0/3 [00:00<?, ?task/s]

Entity Type: SAMPLE_TYPE

NAME CHECKS:
-------------
File name: OK!

CONTENT CHECKS:
-------------
Error: The code should be the same one indicated in the file name

ENTITY CHECKS
-------------
Entity type 'CHEMICAL' already exists.
The set of Property Types assigned to the ('SAMPLE_TYPE') 'CHEMICAL' has been changed compared to the previous version.
The Property type assignment 'BA_DYN_TEST' has been removed.
The label of Property type BAM_FLOOR has been changed compared to the previous version from BAM_FLOOR to BAM Floor.
The description of Property type $NAME has been changed compared to the previous version from Name to Name//Name.
The label of Property type BAM_LOCATION has been changed compared to the previous version from BAM_LOCATION to BAM Location.
The description of Property type BAM_LOCATION has been changed compared to the previous version from BAM Location//BAM Standort/Liegenschaft to BAM Location//BAM Liegenschaft.
The label of Property type BAM_HOUSE has been changed c

## USE THE VISUALIZER

Finally in this section we have the Masterdata Visualizer (which you can execute directly, without running the checker first). 

To run it, just execute the cell below, and it will generate a CSV file *in the same location* where this jupyter notebook is located (check your directory). 

More specifically, a folder with the name of the instance and data will appear (for example, "main_data" if you selected the main instance). Inside this folder, the different CSV files will be generated, once per day of execution (for example, executing it on the 30 of March 2024 in the main instance, the file inside the "main_data" folder will be called "main_30032024.csv).

Also, together with the CSV, we will have listed here all the information regarding the Masterdata content of the instance, if you just need a quick view.

In [24]:
with tqdm(total=100, desc="Generating CSV", unit="percent") as pbar:
    for _ in range(10):
        time.sleep(1)  # Simulate work being done in steps
        pbar.update(10)

    content = generate_csv_and_download()

print(content)

Generating CSV:   0%|          | 0/100 [00:00<?, ?percent/s]

Checking contents in instance: main

Listing SPACES in main

Total number of SPACES: 340

  DEFAULT
  ELN_SETTINGS
  STORAGE
  DEFAULT_LAB_NOTEBOOK
  METHODS
  MATERIALS
  STOCK_CATALOG
  STOCK_ORDERS
  PUBLICATIONS
  8.2_ELN_SETTINGS
  8.2_EQUIPMENT
  8.2_MATERIALS
  8.2_METHODS
  8.2_PUBLICATIONS
  8.2_STORAGE
  8.2_AESKANDA
  8.2_AKOCHERL
  8.2_ASKLODOW
  8.2_CKOEPP
  8.2_CLIAO
  8.2_COEZLUE
  8.2_CSTRANGF
  8.2_CTRUJILL
  8.2_DWU2
  8.2_ENIEDERL
  8.2_ESCHOENS
  8.2_FGOLDMAN
  8.2_FMENSING
  8.2_FMIELENT
  8.2_GWILSCH
  8.2_HSTOLPE
  8.2_JWOESTMA
  8.2_KBORCHAR
  8.2_LPASQUAT
  8.2_MBEHRENS
  8.2_MBUREOS
  8.2_MCLAYTON
  8.2_MGROHMAN
  8.2_MIZANLU
  8.2_MLANGE
  8.2_NASSMANN
  8.2_NEPPLE
  8.2_SKUETTEN
  8.2_SMAACK
  8.2_TKIND
  8.2_TVOELKER
  8.2_UEFFNER
  8.2_VLAY
  8.2_VSUNKAVA
  8.2_WLAI
  8.2_ZNIU
  9.3_ELN_SETTINGS
  9.3_EQUIPMENT
  9.3_MATERIALS
  9.3_METHODS
  9.3_PUBLICATIONS
  9.3_STORAGE
  9.3_AARTINOV
  9.3_AGUMENY1
  9.3_AGUMENYU
  9.3_AJUNEJO
  9.3_APITTNER
  9.3_ASTR